## Grover-based Quantum Classifier
Grover’s algorithm can act as a simple classifier by marking all bitstrings that represent the positive class and then using amplitude amplification to increase their measurement probability. A real-valued feature vector is first mapped to a binary string, an oracle flips the phase of the strings that correspond to cases, and a diffuser performs inversion about the mean.

![GroverAlgorithm](images/GroverAlgorithm.png)

Grover’s algorithm starts with a uniform superposition (H). The Oracle (O) inverts the marked state, and the Diffuser (D) amplifies its probability while suppressing others. A final measurement  reveals the marked state with high probability.

---

### Task
Using the flu screening patient data encoded in binary form — fever, cough, antigen test — develop a Grover-based quantum classifier that marks the flu-positive cases and amplifies their probability. Run the algorithm and display the measurement results as a probability distribution over the positive class bitstrings.

### Expected Output

- Quantum circuit used for the Grover-based classifier.
- A measurement histogram with probability mass concentrated on the flu-positive class bitstrings.

### Experimentation

1. Expand the `patient_data` by including an additional diagnostic test (e.g., blood test result or oxygen saturation), encode it as a binary feature, and observe how the Grover classifier behaves with 4-qubit inputs.  

2. Replace the binary `patient_data` with real-valued feature vectors, and map each sample into a binary string using clinically meaningful thresholds before applying the Grover classifier.  

   Example:  

```
feature_vectors = [
    {"temp": 39.1, "cough": 2.0, "antigen": 0.85},
    {"temp": 38.3, "cough": 7.0, "antigen": 0.12},
    {"temp": 36.8, "cough": 6.0, "antigen": 0.05},
]
```


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer, plot_distribution
from math import floor, sqrt, pi

def oracle(marked):
    n = len(marked[0])
    qc = QuantumCircuit(n)
    for s in marked:
        for i, b in enumerate(s):
            if b == '0':
                qc.x(i)
        qc.h(n-1)
        qc.mcx(list(range(n-1)), n-1)
        qc.h(n-1)
        for i, b in enumerate(s):
            if b == '0':
                qc.x(i)
        qc.barrier() 
    return qc

def diffusion(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    qc.barrier() 
    return qc

def build_classifier(marked):
    n = len(marked[0]) # qubits
    m = len(marked)    # solutions
    k = floor((pi/4) * sqrt((2**n) / m)) # iterations
    grover = QuantumCircuit(n, n)
    grover.h(range(n))  # uniform superposition
    
    O = oracle(marked)
    D = diffusion(n)

    for _ in range(k):
        grover.compose(O, inplace=True)
        grover.compose(D,   inplace=True)

    grover.measure(range(n), range(n))
    return grover

# -----------------------------------------------
#                main program
# -----------------------------------------------

patient_data = {
    '000': 0,  # no fever, no cough, antigen test neg
    '001': 0,  # no fever, no cough, antigen test pos
    '010': 0,  # no fever, cough,    antigen test neg
    '011': 0,  # no fever, cough,    antigen test pos
    '100': 0,  # fever,   no cough,  antigen test neg
    '101': 1,  # fever,   no cough,  antigen test pos  → mark
    '110': 0,  # fever,   cough,     antigen test neg
    '111': 1   # fever,   cough,     antigen test pos  → mark
}

marked = [b for b, y in patient_data.items() if y == 1] 
grover = build_classifier(marked)
display(circuit_drawer(grover, output="mpl"))

# --- Simulate ---
simulator = AerSimulator()
results = simulator.run(grover).result()
counts = results.get_counts(grover)

display(plot_distribution(counts, title="Grover-based quantum classifier (flu example)"))
